In [ ]:
# 1. Import Library
import pandas as pd
import numpy as np

In [ ]:
# 2. Load Dataset
df = pd.read_csv("jabodetabek_house_price.csv")

df.head()

,url,price_in_rp,title,address,district,city,lat,long,facilities,property_type,...,electricity,maid_bedrooms,maid_bathrooms,floors,building_age,year_built,property_condition,building_orientation,garages,furnishing
0,https://www.rumah123.com/properti/bekasi/hos11...,2.990000e+09,Rumah cantik Sumarecon Bekasi\nLingkungan asri...,"Summarecon Bekasi, Bekasi",Summarecon Bekasi,Bekasi,-6.223945,106.986275,"Tempat Jemuran, Jalur Telepon, Taman, Taman",rumah,...,4400 mah,0.0,1.0,2.0,5.0,2017.0,bagus,NaN,0.0,unfurnished
1,https://www.rumah123.com/properti/bekasi/hos10...,1.270000e+09,"Rumah Kekinian, Magenta Summarecon Bekasi","Summarecon Bekasi, Bekasi",Summarecon Bekasi,Bekasi,-6.223945,106.986275,Taman,rumah,...,2200 mah,0.0,0.0,2.0,NaN,NaN,bagus,NaN,0.0,NaN
2,https://www.rumah123.com/properti/bekasi/hos10...,1.950000e+09,Rumah Cantik 2 Lantai Cluster Bluebell Summare...,"Summarecon Bekasi, Bekasi",Summarecon Bekasi,Bekasi,-6.223945,106.986275,"Jogging Track, Kolam Renang, Masjid, Taman,...",rumah,...,2200 mah,1.0,1.0,2.0,NaN,NaN,bagus,NaN,1.0,unfurnished
3,https://www.rumah123.com/properti/bekasi/hos10...,3.300000e+09,Rumah Mewah 2Lantai L10x18 C di Cluster VERNON...,"Summarecon Bekasi, Bekasi",Summarecon Bekasi,Bekasi,-6.223945,106.986275,"Jalur Telepon, Jogging Track, Track Lari, K...",rumah,...,3500 mah,1.0,1.0,2.0,6.0,2016.0,bagus sekali,utara,2.0,unfurnished
4,https://www.rumah123.com/properti/bekasi/hos10...,4.500000e+09,"Rumah Hoek di Cluster Maple Summarecon Bekasi,...","Summarecon Bekasi, Bekasi",Summarecon Bekasi,Bekasi,-6.223945,106.986275,"Jogging Track, Kolam Renang, Taman, Jalur Te...",rumah,...,3500 mah,1.0,1.0,2.0,9.0,2013.0,bagus,utara,1.0,unfurnished


In [ ]:
# 3. Ambil Kolom yang Digunakan
harga = df['price_in_rp']    # harga rumah
luas = df['building_size_m2']   # luas bangunan
kamar = df['bedrooms']       # jumlah kamar

In [ ]:
# 4. Fungsi Keanggotaan (segitiga)
def segitiga(x, a, b, c):
    if a == b and b == c:
        return 1 if x == a else 0

    if x <= a or x >= c:
        return 0
    elif a < x < b:
        return (x - a) / (b - a) if (b - a) != 0 else 0
    elif b <= x < c:
        return (c - x) / (c - b) if (c - b) != 0 else 0
    elif x == b:
        return 1
    else:
        return 0

In [ ]:
# 5. Menentukan Batas Fuzzy
# Harga
h_min, h_max = harga.min(), harga.max()
h_mid = (h_min + h_max) / 2

# Luas bangunan
l_min, l_max = luas.min(), luas.max()
l_mid = (l_min + l_max) / 2

# Jumlah kamar
k_min, k_max = kamar.min(), kamar.max()
k_mid = (k_min + k_max) / 2

In [ ]:
# 6. Defuzzification (Centroid)
def centroid(output):
    x = np.linspace(0, 100, 100)
    return np.sum(x * output) / np.sum(output) if np.sum(output) != 0 else 0

In [ ]:
# 7. Proses Fuzzy untuk Satu Data Rumah
def fuzzy_rumah(h, l, k):
    # Fuzzifikasi harga
    murah = segitiga(h, h_min, h_min, h_mid)
    sedang_h = segitiga(h, h_min, h_mid, h_max)
    mahal = segitiga(h, h_mid, h_max, h_max)

    # Fuzzifikasi luas
    sempit = segitiga(l, l_min, l_min, l_mid)
    sedang_l = segitiga(l, l_min, l_mid, l_max)
    luas_b = segitiga(l, l_mid, l_max, l_max)

    # Fuzzifikasi kamar
    sedikit = segitiga(k, k_min, k_min, k_mid)
    sedang_k = segitiga(k, k_min, k_mid, k_max)
    banyak = segitiga(k, k_mid, k_max, k_max)

    # Rule base (MIN untuk AND)
    r1 = min(murah, luas_b, banyak)     # sangat layak
    r2 = min(murah, sedang_l, sedang_k) # layak
    r3 = min(sedang_h, luas_b, banyak)  # layak
    r4 = min(mahal, sempit, sedikit)    # tidak layak
    r5 = min(mahal, sedang_l, sedang_k) # tidak layak

    # Output fuzzy
    x = np.linspace(0, 100, 100)
    tidak_layak = np.maximum(r4, r5) * np.maximum(0, np.minimum(1, (50 - x) / 50))
    layak = np.maximum(r2, r3) * np.maximum(0, np.minimum((x - 30) / 20, (70 - x) / 20))
    sangat_layak = r1 * np.maximum(0, np.minimum((x - 60) / 40, 1))

    output = np.maximum.reduce([tidak_layak, layak, sangat_layak])

    return centroid(output)

In [ ]:
# 8. Hitung Skor Kelayakan Semua Rumah
skor = []

for i in range(len(df)):
    s = fuzzy_rumah(harga[i], luas[i], kamar[i])
    skor.append(s)

df['skor_kelayakan'] = skor

In [ ]:
# 9. Mengambil 5 Rumah Terbaik
top5 = df.sort_values(by='skor_kelayakan', ascending=False).head(5)

top5[['price_in_rp', 'building_size_m2', 'bedrooms', 'skor_kelayakan']]

,price_in_rp,building_size_m2,bedrooms,skor_kelayakan
1257,470000000.0,40.0,2.0,50.0
1258,470000000.0,40.0,2.0,50.0
3282,630000000.0,40.0,2.0,50.0
1260,400000000.0,40.0,2.0,50.0
3277,630000000.0,40.0,2.0,50.0


In [ ]:
# 10. Menyimpan Output ke file
top5.to_csv('top_5_rumah_terbaik.csv', index=False)
print("File berhasil disimpan!")

File berhasil disimpan!
